**A Python script named `USDA_food_tool**

**Status Broken**

This script prompts the user for an API key. API keys can be obtained from FoodData Central. After the API key is entered, the script authenticates and then test the connection. The script connects to the USDA databases via API to search FoodData central 

Rate Limiting: Exceeding the quota (1,000 requests per hour)
show current rate for given API key?

The script offers a choice between USDA Foundational Food data, Branded Foods data, or both. This food data is located in two distinct tables: "Foundation Foods" and "Branded Foods.”

Food search will use a keyword text string of 200 characters or less. The data that should be returned is as follows.
The data for “Foundational Foods” will be returned in a table with the following columns: ‘Data Type’, ‘Food Category’, ‘FDC ID’, ‘NDB Number’, and ‘FDC Published’.

The Branded Foods data, returned in tabular form, includes 'Data Type', 'Food Category', 'Brand', 'FDC ID', 'GTIN/UPC', 'FDC Published', 'Market Country', and 'Package Weight'.


In [1]:
import requests
import getpass
import sys
import pandas as pd

API_BASE = "https://api.nal.usda.gov/fdc/v1"

def prompt_api_key():
    return getpass.getpass("Enter your FoodData Central API Key: ")

def test_api_key(api_key):
    resp = requests.get(f"{API_BASE}/foods/list?api_key={api_key}&pageSize=1")
    if resp.status_code == 200:
        print("API Key is valid.")
        return True
    elif resp.status_code == 401:
        print("Invalid API key. Please check and try again.")
        return False
    else:
        print(f"Error: Received status code {resp.status_code}")
        return False

def choose_data_type():
    print("Choose data type to search:")
    print("1. Foundation Foods")
    print("2. Branded Foods")
    print("3. Both")
    choice = input("Select (1/2/3): ").strip()
    if choice not in {"1", "2", "3"}:
        print("Invalid selection. Exiting.")
        sys.exit(1)
    return choice

def search_food(api_key, query, data_type):
    results = []
    payload = {
        "api_key": api_key,
        "query": query,
        "requireAllWords": True,
        "pageSize": 25,
    }

    if data_type in ("1", "3"):
        # Foundation Foods search
        payload_ff = {**payload, "dataType": ["Foundation"]}
        resp = requests.post(f"{API_BASE}/foods/search", json=payload_ff)
        if resp.status_code == 200:
            ff = resp.json().get('foods', [])
            results.append(("Foundation", ff))
        else:
            print(f"Error searching Foundation Foods: {resp.status_code}")
            results.append(("Foundation", []))

    if data_type in ("2", "3"):
        # Branded Foods search
        payload_bf = {**payload, "dataType": ["Branded"]}
        resp = requests.post(f"{API_BASE}/foods/search", json=payload_bf)
        if resp.status_code == 200:
            bf = resp.json().get('foods', [])
            results.append(("Branded", bf))
        else:
            print(f"Error searching Branded Foods: {resp.status_code}")
            results.append(("Branded", []))

    return results

def format_foundation(foods):
    rows = []
    for food in foods:
        row = [
            food.get("Description"),
            food.get("dataType"),
            food.get("foodCategory"),
            food.get("fdcId"),
            food.get("ndbNumber"),
            food.get("publishedDate"),
        ]
        rows.append(row)
    return pd.DataFrame(rows, columns=[
        "Description", "Data Type", "Food Category", "FDC ID", "NDB Number", "FDC Published"
    ])

def format_branded(foods):
    rows = []
    for food in foods:
        row = [
            food.get("Description"),
            food.get("dataType"),
            food.get("foodCategory"),
            food.get("brandOwner"),
            food.get("fdcId"),
            food.get("gtinUpc"),
            food.get("publishedDate"),
            food.get("marketCountry"),
            food.get("packageWeight"),
        ]
        rows.append(row)
    return pd.DataFrame(rows, columns=[
        "Description", "Data Type", "Food Category", "Brand", "FDC ID", "GTIN/UPC",
        "FDC Published", "Market Country", "Package Weight"
    ])

def main():
    print("=== USDA FoodData Central Search Tool ===")
    api_key = prompt_api_key()
    if not test_api_key(api_key):
        sys.exit(1)
    data_type = choose_data_type()
    query = input("Enter food search keywords (max 200 chars): ").strip()[:200]
    results = search_food(api_key, query, data_type)
    for dtype, foods in results:
        if dtype == "Foundation":
            df = format_foundation(foods)
            print("\n--- Foundation Foods Results ---")
        elif dtype == "Branded":
            df = format_branded(foods)
            print("\n--- Branded Foods Results ---")
        else:
            continue
        if df.empty:
            print("No results found.")
        else:
            print(df.to_string(index=False))

if __name__ == "__main__":
    main()


=== USDA FoodData Central Search Tool ===


Enter your FoodData Central API Key:  ········


API Key is valid.
Choose data type to search:
1. Foundation Foods
2. Branded Foods
3. Both


Select (1/2/3):  3
Enter food search keywords (max 200 chars):  Tomato


Error searching Foundation Foods: 403
Error searching Branded Foods: 403

--- Foundation Foods Results ---
No results found.

--- Branded Foods Results ---
No results found.


**ERROR Possibilities**

USDA API 403 Error: Cause and Solution
A 403 error when using the USDA FoodData Central API typically means the client is forbidden from accessing the resource. Here are the most common causes and the steps to fix them:

Common Causes
Incorrect API Key Usage: Passing the API key as a query parameter is required by the USDA API, not as an authentication header or within the JSON body.

Malformed Request JSON: The API expects the API key in the query string and will respond with a 403 if this is moved, misspelled, or omitted.

Rate Limiting: Exceeding the quota (1,000 requests per hour) can temporarily block your key for one hour.

Endpoint/Method Confusion: The endpoint /foods/search expects the API key as a query parameter in both POST and GET requests, and some parameters may need to match exactly ("dataType" not "dataTypes", etc.).